In [14]:
import requests
import json
import os
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


In [24]:



congress_url = "https://dadosabertos.camara.leg.br/api/v2/"
current_year = datetime.now().year
years = list(range(current_year, 2007, -4))

def get_all_congress_members_year(year):
    print(f'------------{year}------------')
    try:
        if not os.path.exists(f'data/deputados-{year}.json'):
            all_members_response = requests.get(f'{congress_url}/deputados?dataInicio={year}-01-01&dataFim={year+3}-01-01')

            if(all_members_response.status_code == 200):
                with open(f'data/deputados-{year}.json', 'w', encoding='utf-8') as file:
                    json.dump(all_members_response.json(), file, ensure_ascii=False)
        else:
            print(f'File for year {year} already exists. Exiting')
    except:
        print(f'Error at request')


In [25]:
for year in years:
    get_all_congress_members_year(year)

------------2025------------
------------2021------------
------------2017------------
------------2013------------
------------2009------------


KeyError: 'siglaPartido'

In [ ]:
partidos_serie_historica = []

for year in years:
    
    with open(f'data/deputados-{year}.json', 'r', encoding='utf-8') as file:
        dado_ano = {}
        response = json.load(file)
        data = response['dados']
        
        for deputado in data:
            partido = deputado['siglaPartido']
            if partido not in dado_ano:
                dado_ano[partido] = 1
            else:
                dado_ano[partido] +=1
        partidos_serie_historica.append(dado_ano)
        

for partido in partidos_serie_historica:
    print(partido)
    partidos_serie_historica_ordenado = dict(sorted(partido.items(), key=lambda item: item[1], reverse=True)[:10])



{'PL': 96, 'MDB': 47, 'REPUBLICANOS': 46, 'PSDB': 13, 'NOVO': 4, 'PP': 52, 'PDT': 20, 'PT': 73, 'CIDADANIA': 5, 'UNIÃO': 66, 'PCdoB': 8, 'PV': 5, 'AVANTE': 7, 'PSD': 49, 'SOLIDARIEDADE': 5, 'PSB': 15, 'PODE': 15, 'PSOL': 13, 'S.PART.': 1, 'PRD': 5, 'PSC': 1, 'REDE': 1}
{'PL': 142, 'PSC': 16, 'PSL': 49, 'UNIÃO': 133, 'PROS': 11, 'MDB': 67, 'REPUBLICANOS': 77, 'PSDB': 34, 'NOVO': 8, 'PP': 90, 'PR': 4, 'PT': 82, 'PDT': 31, 'DEM': 24, 'PATRIOTA': 7, 'PSB': 37, 'CIDADANIA': 12, 'PSD': 80, 'PCdoB': 10, 'PV': 11, 'AVANTE': 10, 'SOLIDARIEDADE': 13, 'PSOL': 16, 'PODE': 18, 'PTB': 7, 'S.PART.': 5, 'PRB': 3, 'PPS': 1, 'REDE': 1, 'PHS': 1}
{'DEM': 49, 'PHS': 15, 'PR': 63, 'PL': 30, 'PSL': 60, 'PROS': 14, 'PP**': 44, 'PODE': 28, 'PTB': 16, 'PTdoB': 3, 'AVANTE': 12, 'PT': 61, 'PTN': 12, 'PMDB': 51, 'MDB': 63, 'PSDB': 55, 'PSB': 56, 'S.PART.': 12, 'PRB': 37, 'NOVO': 5, 'PP': 72, 'PDT': 33, 'PRP': 2, 'PATRI': 8, 'PATRIOTA': 4, 'REDE': 4, 'PPS': 16, 'CIDADANIA': 8, 'PSD': 58, 'PCdoB': 14, 'REPUBLICANOS

In [ ]:
df = pd.DataFrame(partidos_serie_historica[:10], index=years)


plt.figure(figsize=(20, 10))

for partido in  df.columns:
    
    plt.plot(df.index, df[partido], marker='o', linestyle='-', label=partido)

plt.xlabel("Ano")
plt.ylabel("Número de Deputados")
plt.title("Variação do Número de Deputados por Partido (2008-2025)")
plt.legend()
plt.grid(True)

# Exibir o gráfico
plt.show()

IndexError: single positional indexer is out-of-bounds

In [42]:


df_variacao = df.pct_change() * 100
df_variacao.head()




/tmp/ipykernel_35452/367321092.py:1: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_variacao = df.pct_change() * 100


,PL,MDB,REPUBLICANOS,PSDB,NOVO,PP,PDT,PT,CIDADANIA,UNIÃO,...,PATRI,SD,PTC,PEN,PMN,PSDC,DC,PMB,SDD,PRTB
2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021,47.916667,42.553191,67.391304,161.538462,100.0,73.076923,55.000000,12.328767,140.000000,101.515152,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017,-78.873239,-5.970149,-74.025974,61.764706,-37.5,-20.000000,6.451613,-25.609756,-33.333333,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013,0.000000,0.000000,0.000000,58.181818,0.0,0.000000,9.090909,106.557377,0.000000,0.000000,...,0.0,176.923077,0.0,100.0,200.0,200.0,0.0,NaN,NaN,NaN
2009,0.000000,0.000000,0.000000,12.643678,0.0,0.000000,25.000000,23.015873,0.000000,0.000000,...,0.0,0.000000,300.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
